# Next

1. Split up column 40 into 1 or 2 columns: "TotalCost" and "CostPerKg". However - that would result in effectively two different data sets.
2. Convert "TotalCost" units into millions/thousands of shillings? Depends what the magnitude is. Otherwise, leave as raw numbers. 

## Main Income cols to clean

1. ~~Q38.CashCropEarnings~~
2. ~~Q61.CropEarning~~
3. ~~Q69.AmountEarnedCattle~~
4. ~~Q72.TotalEarningCattle [note I don't know the difference between Q69 and Q72; some people have given same answers for both, some people haven't...]~~
5. ~~Q113.CostOfChange~~
6. ~~Q118.CostStarting~~

In [ ]:
import csv
import pandas as pd
import numpy as np
import re

df = pd.read_csv('DataPrep/UgandaClean.csv')
df2 = pd.read_csv('DataPrep/KenyaClean.csv')

In [ ]:
df.columns[0:20]

In [ ]:
df2.columns[0:20]

In [ ]:
# Export unique values

target_cols = ['Q21','Q93', 'Q28', 'Q95', 'Q97', 'Q99', 'Q147', 'Q148', 'Q150', 'Q162', 'Q163', 'Q165', 'Q166', 'Q170', 'Q171']


find_cols = [col for target in target_cols for col in df.columns if target in col and not '.FreeText' in col and not '.Clean' in col]
find_cols

In [ ]:
# Cols with nothing in them
'''


'''

def build_dataframe_from_uniques(source_dataframe, cols_to_export, target_file):
    uniques_dict = { col: pd.Series(pd.unique(source_dataframe[col].dropna())) for col in cols_to_export }
    uniques_df = pd.DataFrame.from_dict(uniques_dict)
    uniques_df = uniques_df.replace('-1', np.nan)
    uniques_df.to_csv(target_file)
    print 'exported to ' + target_file

cols_to_export = ['Q21.Income.Other.O6.Text',
                  'Q28.FarmHealth.Text',
  'Q93.BankAccountChallenges',
  'Q97.BankCreditChallenges', 
 'Q148.IdeasOfChanges', 
 'Q95.BankLoanFacilityChallenges',
 'Q99.SACCOChallenges',
 'Q147.OtherInfoAreas',
 'Q150.HelpToImplementChanges',
 'Q162.MainChallengesOnFarm',
 'Q163.SolutionsToAddressProblems',

 'Q166.ShareNewFarmingPractice.Example',
 'Q170.ChallengesOfSharingInfo',
 'Q171.MakeSharingEasier']



build_dataframe_from_uniques(df, cols_to_export,'DataPrep/Uganda_UniqueNarrativeDataValues.csv' )
build_dataframe_from_uniques(df2, cols_to_export,'DataPrep/Kenya_UniqueNarrativeDataValues.csv' )


In [ ]:
u_df = pd.read_csv('DataPrep/Uganda_UniqueNarrativeDataValues.csv')
u_df

In [ ]:
def compile(expression):
    return re.compile(expression, re.I)

def clean_amount_columns (row, column_name):
    input_value = row[column_name]
     
    if pd.isnull(input_value):
        return input_value 
    
    # global cleanup
    input_value = re.sub('\s', '', input_value)
    leading_string_pattern = compile('v')
    input_value = leading_string_pattern.sub('', input_value)
    
    commas_pattern = compile('^[\d\,]+$')
    commas_pattern_with_random_end_string = compile('^[\d\,]+[^\d]+$')
    m_pattern = compile('^\d{1,}(\.\d{1,})?m(illion|ilion)?(ugx)?')
    commas_and_currency_pattern = compile('^[\d\,]+(ugx|/=|shs)$')
    commas_and_currency_with_random_end_string = compile('^[\d\,]+(ugx|/=)[^\d]+$')
    no_numbers_pattern = compile('^[^\d]+$')
    

    replace_million = compile('m(illion|ilion)?(ugx)?')
    replace_comma = compile('(\,)')
    replace_currency = compile('(ugx|/=|shs)')
    replace_currency_with_random_string = compile('(ugx|/=|shs)[^\d]+')
    replace_random_end_string = compile('[^\d]+$')
    
    if commas_pattern.search(input_value):
        return float(replace_comma.sub('', input_value))
    elif m_pattern.search(input_value):
        number_only = float(replace_million.sub('', input_value)) * 1000000
        return number_only
    elif commas_and_currency_pattern.search(input_value):
        stripped_of_commas = replace_comma.sub('', input_value)
        return float(replace_currency.sub('', stripped_of_commas))
    elif commas_and_currency_with_random_end_string.search(input_value):
        stripped_of_commas = replace_comma.sub('', input_value)
        return float(replace_currency_with_random_string.sub('', stripped_of_commas))
    elif commas_pattern_with_random_end_string.search(input_value):
        stripped_of_commas = replace_comma.sub('', input_value)
        return float(replace_random_end_string.sub('', stripped_of_commas))
    elif re.search(no_numbers_pattern, input_value):
        return np.nan
    else:
        return input_value
    
def keep_free_text_only (row, column_name): 
    input_value = row[column_name]   
    if pd.isnull(input_value):
        return input_value 
    no_numbers_pattern = compile('^[^\d]*$')
    if no_numbers_pattern.search(input_value):
        return input_value
    else:
        return np.nan

def clean_numeric_columns_in(dataframe, cols_to_clean):
    for col_name in cols_to_clean:
        clean_col_name = col_name+'.Clean'
        freetext_col_name = col_name+'.FreeText'
        if ~(dataframe[clean_col_name].apply(np.isreal).all(skipna=True)):
            print('cleaning ' + col_name)
            dataframe[freetext_col_name] = dataframe.apply(lambda row:keep_free_text_only(row, col_name), axis=1)
            dataframe[clean_col_name] = dataframe.apply(lambda row: clean_amount_columns(row, col_name), axis=1)
        else:
            print('skipping, already clean: ' + col_name)
        print(col_name, dataframe[clean_col_name].apply(np.isreal).all(skipna=True))



In [ ]:
target_df = df2

# Q38, Q61, Q69, Q72, Q113, Q118
target_cols = ['Q38', 'Q61', 'Q69', 'Q72', 'Q113', 'Q118']

cols_to_clean = [col for target in target_cols for col in target_df.columns if target in col and not '.FreeText' in col and not '.Clean' in col]

print(cols_to_clean)


clean_numeric_columns_in(df, cols_to_clean)

[col for target in target_cols for col in target_df.columns if target in col]
#pd.unique(df[cols_to_clean[0]])

In [ ]:
# Use for investigating false columns

pd.unique(df2['Q69.AmountEarnedCattle'])
df2['Q69.AmountEarnedCattle']


In [ ]:
#df.to_csv('DataPrep/UgandaClean.csv')